In [ ]:
from __init__ import *

In [ ]:
plot = False

In [ ]:
signal_directory_path = "{}/signal/".format(SETTING["output_directory_path"])

kraft.path(signal_directory_path, "directory")

In [ ]:
def compute_surprise_(r, d):

    assert (0 <= r).all()

    assert (0 <= d).all()

    r_sum = r.sum()

    assert 0 < r_sum

    d_sum = d.sum()

    assert 0 < d_sum

    rp = r / r_sum

    dp = d / d_sum

    s = rp * np.log(dp / rp)

    s[s == -np.inf] = 0

    return s

In [ ]:
def compute_surprise(r, d, plot=True):

    assert (r.index == d.index).all()

    return pd.Series(compute_surprise_(r.values, d.values), index=d.index, name=d.name)

In [ ]:
for data_dict_index, data_dict in enumerate(SETTING["model_data_dicts"]):

    print("=" * 8, data_dict["name"], "=" * 8)

    dataframe = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

    dataframe.columns.name = data_dict["axis1_name"]

    dataframe = dataframe.loc[(dataframe != 0).any(axis=1)]

    sum_axis = dataframe.sum()

    surprise = dataframe.apply(
        lambda series: compute_surprise(sum_axis, series, plot=False), axis=1
    )

    kraft.summarize_dataframe(surprise, plot=plot)

    surprise_negative = surprise.clip(upper=0)

    surprise_positive = surprise.clip(lower=0)

    surprise_negative_elements = kraft.select_index(
        surprise_negative.sum(axis=1), "<", standard_deviation=2
    )

    surprise_positive_elements = kraft.select_index(
        surprise_positive.sum(axis=1), ">", standard_deviation=1.5
    )

    surprise_negative_select = surprise_negative.loc[surprise_negative_elements]

    surprise_positive_select = surprise_positive.loc[surprise_positive_elements]

    surprise_negative_select.index = pd.Index(
        ("(-) {}".format(element) for element in surprise_negative_select.index),
        name=surprise_negative_select.index.name,
    )

    surprise_positive_select.index = pd.Index(
        ("(+) {}".format(element) for element in surprise_positive_select.index),
        name=surprise_positive_select.index.name,
    )

    signal = pd.concat((-surprise_negative_select, surprise_positive_select))

    signal.to_csv("{}/{}.tsv".format(signal_directory_path, data_dict_index), sep="\t")